In [39]:
import sympy as sp
import numpy as np
import math

def matriz_DH(theta, d, a, alpha):
    """
    Gera a matriz de Denavit-Hartenberg para uma junta específica.

    Args:
        theta (sympy.Symbol): Símbolo theta representando o ângulo.
        d (float): Valor da variável d.
        a (float): Valor da variável a.
        alpha (float): Valor do ângulo alpha em radianos.

    Returns:
        numpy.ndarray: Matriz de Denavit-Hartenberg gerada.
    """

    # Converte alpha de graus para radianos
    alpha = sp.rad(alpha)

    cos_theta = sp.cos(theta)
    sin_theta = sp.sin(theta)
    cos_alpha = sp.cos(alpha)
    sin_alpha = sp.sin(alpha)

    matriz = np.array([[cos_theta, -sin_theta*cos_alpha, sin_theta*sin_alpha, a*cos_theta],
                       [sin_theta, cos_theta*cos_alpha, -cos_theta*sin_alpha, a*sin_theta],
                       [0, sin_alpha, cos_alpha, d],
                       [0, 0, 0, 1]])

    return matriz

def matriz_DH_total(parametros):
    """
    Gera a matriz de Denavit-Hartenberg para um conjunto de juntas.

    Args:
        parametros (list): Lista de dicionários contendo os parâmetros de cada junta.

    Returns:
        numpy.ndarray: Matriz de Denavit-Hartenberg total gerada.
    """
    matriz_total = np.eye(4)

    for param in parametros:
        d = param['d']
        a = param['a']
        alpha = param['alpha']
        theta = param['theta']

        matriz = matriz_DH(theta, d, a, alpha)
        matriz_total = np.dot(matriz_total, matriz)

    return matriz_total

# Exemplo de uso
thetas = [sp.Symbol('theta1'), sp.Symbol('theta2'), sp.Symbol('theta3'), sp.Symbol('theta4'), sp.Symbol('theta5'), sp.Symbol('theta6')]
ds = [495, 0, 0, 1475, 0, 135]
as_ = [175, 1095, 175, 0, 0, 0]
alphas = [-90, 0, -90, 90, 90, 0]

parametros_juntas = [
    {'theta': thetas[0], 'd': ds[0], 'a': as_[0], 'alpha': alphas[0]},
    {'theta': thetas[1], 'd': ds[1], 'a': as_[1], 'alpha': alphas[1]},
    {'theta': thetas[2], 'd': ds[2], 'a': as_[2], 'alpha': alphas[2]},
    {'theta': thetas[3], 'd': ds[3], 'a': as_[3], 'alpha': alphas[3]},
    {'theta': thetas[4], 'd': ds[4], 'a': as_[4], 'alpha': alphas[4]},
    {'theta': thetas[5], 'd': ds[5], 'a': as_[5], 'alpha': alphas[5]}
]

matriz_total_DH = matriz_DH_total(parametros_juntas)
# Simplifica a matriz de Denavit-Hartenberg
matriz_total_DH = sp.simplify(matriz_total_DH)


In [49]:
print("Matriz de Denavit-Hartenberg:")
# Considera a notação abaixo para expressar a matriz de Denavit-Hartenberg
# [ [a, b, c, x],
#   [d, e, f, y],
#   [g, h, i, z],
#   [0, 0, 0, 1] ]

ref = ['A', 'B', 'C', 'D', 'E', 'F']

# Imprime cada termo da matriz de Denavit-Hartenberg em latex
for i in range(4):
    for j in range(4):
        # print(f"{sp.latex(matriz_total_DH[i, j])}", end='\n------------------------------------------------\n')

        #Substitui os onde há '\left(' por '(' e '\right)' por ')'
        backslash_char = "\\"
        latex_str = sp.latex(matriz_total_DH[i, j])
        latex_str = latex_str.replace(backslash_char + "left(", "(")
        latex_str = latex_str.replace(backslash_char + "right)", ")")

        # Imprime o termo da matriz de Denavit-Hartenberg
        print(f"{latex_str}", end='\n------------------------------------------------\n')

Matriz de Denavit-Hartenberg:
(((- 1.0 \sin{(\theta_{2} )} \sin{(\theta_{3} )} \cos{(\theta_{1} )} + 1.0 \cos{(\theta_{1} )} \cos{(\theta_{2} )} \cos{(\theta_{3} )}) \cos{(\theta_{4} )} + 1.0 \sin{(\theta_{1} )} \sin{(\theta_{4} )}) \cos{(\theta_{5} )} + (- 1.0 \sin{(\theta_{2} )} \cos{(\theta_{1} )} \cos{(\theta_{3} )} - 1.0 \sin{(\theta_{3} )} \cos{(\theta_{1} )} \cos{(\theta_{2} )}) \sin{(\theta_{5} )}) \cos{(\theta_{6} )} + ((- 1.0 \sin{(\theta_{2} )} \sin{(\theta_{3} )} \cos{(\theta_{1} )} + 1.0 \cos{(\theta_{1} )} \cos{(\theta_{2} )} \cos{(\theta_{3} )}) \sin{(\theta_{4} )} - 1.0 \sin{(\theta_{1} )} \cos{(\theta_{4} )}) \sin{(\theta_{6} )}
------------------------------------------------
- (((- 1.0 \sin{(\theta_{2} )} \sin{(\theta_{3} )} \cos{(\theta_{1} )} + 1.0 \cos{(\theta_{1} )} \cos{(\theta_{2} )} \cos{(\theta_{3} )}) \cos{(\theta_{4} )} + 1.0 \sin{(\theta_{1} )} \sin{(\theta_{4} )}) \cos{(\theta_{5} )} + (- 1.0 \sin{(\theta_{2} )} \cos{(\theta_{1} )} \cos{(\theta_{3} )} - 1

In [37]:
# Função que cria a matriz de Denavit-Hartenberg parcial da junta j até a junta K
def matriz_DH_parcial(parametros, j, k):
    """
    Gera a matriz de Denavit-Hartenberg para um conjunto de juntas.

    Args:
        parametros (list): Lista de dicionários contendo os parâmetros de cada junta.
        j (int): Número da junta inicial.
        k (int): Número da junta final.

    Returns:
        numpy.ndarray: Matriz de Denavit-Hartenberg parcial gerada.
    """
    matriz_total = np.eye(4)

    for i in range(j, k+1):
        param = parametros[i]
        d = param['d']
        a = param['a']
        alpha = param['alpha']
        theta = param['theta']

        matriz = matriz_DH(theta, d, a, alpha)
        matriz_total = np.dot(matriz_total, matriz)

    return matriz_total

matriz_parcial_DH = matriz_DH_parcial(parametros_juntas, 3, 3)

# Simplifica a matriz
matriz_parcial_DH = sp.nsimplify(matriz_parcial_DH, tolerance=1e-1, rational=True)


print("Matriz de Denavit-Hartenberg parcial:")
# print in latex
print('\\begin{bmatrix}')
for i in range(4):
    for j in range(4):
        print(sp.latex(matriz_parcial_DH[i, j]), end=' & ')
    print('\\\\')
print('\\end{bmatrix}')

Matriz de Denavit-Hartenberg parcial:
\begin{bmatrix}
\cos{\left(\theta_{4} \right)} & 0 & \sin{\left(\theta_{4} \right)} & 0 & \\
\sin{\left(\theta_{4} \right)} & 0 & - \cos{\left(\theta_{4} \right)} & 0 & \\
0 & 1 & 0 & 1475 & \\
0 & 0 & 0 & 1 & \\
\end{bmatrix}


In [36]:
# Aplica valores de theta na matriz final
# Usando loop for

new_theta = [sp.rad(0), sp.rad(-3.651 - 90), sp.rad( 3.565), sp.rad(0), sp.rad(0-180), sp.rad(0)]

matriz_final = matriz_total_DH
for i in range(0,4):
    for j in range(0,4):
        matriz_final[i][j] = matriz_final[i][j].subs({thetas[0]: new_theta[0], thetas[1]: new_theta[1], thetas[2]: new_theta[2], thetas[3]: new_theta[3], thetas[4]: new_theta[4], thetas[5]: new_theta[5]})

matriz_final = sp.nsimplify(matriz_final, tolerance=1e-1, rational=True)

print('Posição final do efetuador:')
print('X: ', math.trunc(matriz_final[0][3]))
print('Y: ', math.trunc(matriz_final[1][3]))
print('Z: ', math.trunc(matriz_final[2][3]))

Posição final do efetuador:
X:  1785
Y:  0
Z:  1765
